# MAST30034_Applied Data Science_Project1

## Import Libraries

In [1]:
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium
import os

In [2]:
# Create a spark session
spark = (
SparkSession.builder.appName("ADS project 1")
.config("spark.sql.repl.eagerEval.enabled", True)
.config("spark.driver.memory","20G")
.config("spark.executor.memory","20G")
.config("spark.sql.parquet.cacheMetadata", "true")
.config("spark.sql.session.timeZone", "Etc/UTC")
.getOrCreate()
)

24/08/24 11:44:37 WARN Utils: Your hostname, chumuhandeMacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.13.153.226 instead (on interface en0)
24/08/24 11:44:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 11:44:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read data

Read the raw data of yellow taxi and green taxi

In [3]:
taxi_yellow_raw = spark.read.parquet('../data/raw/NYCTLC_data/yellow_taxi/2023/*.parquet')
taxi_green_raw = spark.read.parquet('../data/raw/NYCTLC_data/green_taxi/2023/*.parquet')

Read the raw data of cityBike

In [5]:
cityBike_data_dir = '../data/raw/external_data/cityBike_data/citybike_raw_data'
cityBike_file = []

# Read all the csv file in the cityBike_data_dir
for root, dirs, files in os.walk(cityBike_data_dir):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df_spark = spark.read.csv(file_path, header=True, inferSchema=True)
            cityBike_file.append(df_spark)

# Merge all the data together
citybike_raw_data = cityBike_file[0]
for data in cityBike_file[1:]:
    citybike_raw_data = citybike_raw_data.union(data)

Read the raw weather data of New York

In [3]:
weather_raw = spark.read.csv('../data/raw/external_data/weather_raw_data.csv',header=True)

## Data Exploration

### Cursory look at the data

In [4]:
taxi_yellow_raw.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2023-10-01 00:16:44|  2023-10-01 00:16:49|              1|          0.0|         1|                 N|         168|         168|           2|        3.0|  1.0|    0.5|       0.

In [7]:
taxi_green_raw.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2023-03-01 00:25:10|  2023-03-01 00:35:47|                 N|         1|          82|         196|              1|         2.36|       13.5|  1.0|    0.

In [8]:
citybike_raw_data.show()

+----------------+-------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|          started_at|            ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|5772778972800B87|electric_bike|2023-12-20 06:18:...|2023-12-20 06:24:...|     W 41 St & 8 Ave|         6602.03|Madison Ave & E 2...|       6131.12|      40.75633049|     -73.989260077|40.742684734875695|-73.98671329021454|       member|
|32DD834BEA2ABA5B| classic_bike|2023-12-21 16:15

In [9]:
weather_raw.show()

+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-----------+--------------------+
|     name|           datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|          conditions|       icon|            stations|
+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-----------+--------------------+
| new York|2023-01-01T00:00:00|12.2|     12.2|  11|   92.43|  0.22|       100|      rain|   0|        0|     8.1|      1.9|    295|          1007.1|       100|       7.7|             0|  

### view the shape of data

In [10]:
print(taxi_yellow_raw.count())
print(len(taxi_yellow_raw.columns))

35243460
19


In [11]:
print(taxi_green_raw.count())
print(len(taxi_green_raw.columns))

718849
20


In [12]:
print(citybike_raw_data.count())
print(len(citybike_raw_data.columns))

33311618
13


In [13]:
print(weather_raw.count())
print(len(weather_raw.columns))

8760
24


### view the sturcture of data

In [16]:
taxi_yellow_raw.describe()

summary,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,35243460,34005847,35243460,34005847,34005847,35243460,35243460,35243460,35243460,35243460,35243460,35243460,35243460,35243460,35243460,34005847,34005847
mean,1.7396488596749582,1.3711210898525774,4.109969790423746,1.6546293053662213,NULL,165.0690936417707,163.91144076092417,1.1837233063950021,19.622770646514084,1.557641658338879,0.4853840511118939,3.5356942161748766,0.5959356771380228,0.9792193558747485,28.58738102525042,2.2637631478492506,0.14294290184861444
stddev,0.4443954955468373,0.8922777526110512,240.51214168146072,7.505324423430077,NULL,63.97505405256568,69.8476148622298,0.5585367121966807,78.7775916553566,2.4996453565931813,0.11007453318210894,4.173499419113611,2.216133960220088,0.2004600212655753,80.14649292985101,0.7993326493697479,0.47720752290181095
min,1,0,0.0,1,N,1,1,0,-1087.3,-39.17,-0.5,-411.0,-91.3,-1.0,-1094.05,-2.5,-1.75
max,6,9,345729.44,99,Y,265,265,5,386983.63,10002.5,52.09,4174.0,665.56,1.0,386987.63,2.75,1.75


In [17]:
taxi_green_raw.describe()

summary,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,718849,667560,667560,718849,718849,667560,718849,718849,718849,718849,718849,718849,0,718849,718849,667560,667525,667560
mean,1.869570660875928,NULL,1.1714138055006291,98.299194963059,139.46967026454791,1.291290670501528,20.054807532597074,18.494785956438527,0.8765909808596799,0.5677396087356316,2.3487429070638703,0.24965746631073524,NULL,0.9791685040947581,24.052933369873095,1.3512373419617711,1.0365709149470057,0.7441106417400684
stddev,0.33677542130701305,NULL,1.060129298224493,59.914941795206154,76.31171713439623,0.925739142987554,1073.185655902081,18.917444120732444,1.3410976056903952,0.385989280208792,3.379316116629143,1.3621565732366472,NULL,0.15139163606499412,20.75873460013295,0.5068659967853087,0.18770598261454347,1.2216929902970093
min,1,N,1,1,1,0,0.0,-500.0,-5.0,-0.5,-22.2,-6.55,NULL,-1.0,-501.0,1,1,-2.75
max,2,Y,99,265,265,9,278990.28,4003.0,12.7,4.25,460.0,65.0,NULL,1.0,4004.5,5,2,2.75


In [18]:
citybike_raw_data.describe()

summary,ride_id,rideable_type,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
count,33311618,33311618,33293477,33293477,33215992,33215992,33311618,33311618,33286571,33286571,33311618
mean,Infinity,NULL,NULL,6001.77877202791,NULL,5997.021505191426,40.73985634693476,-73.97268450765375,40.7396818599351,-73.97273256445152,NULL
stddev,NaN,NULL,NULL,1130.1617108814019,NULL,1129.21928065291,0.040736961895780785,0.027938237128016928,0.04836617111718019,0.05217842995813764,NULL
min,00000045A35269CB,classic_bike,1 Ave & E 110 St,2733.03,1 Ave & E 110 St,190 Morgan,40.574825525,-74.072216153,-37.26,-173.37,casual
max,FFFFFFC6B10A98E2,electric_bike,York St & Marin Blvd,SYS039,York St & Marin Blvd,Shop Morgan,40.894158244,-73.79639436666666,41.17,0.0,member


In [19]:
weather_raw.describe()

summary,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
count,8760,8760,8760,8760,8760,8760,8760,8760,994,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760,8760
mean,NULL,NULL,14.599520547945412,13.890776255707962,7.082351598173469,63.255827625570554,0.04791210045662115,9.97716894977169,NULL,0.001381278538812...,0.02305936073059361,15.499086757990758,6.525867579908609,190.3936073059361,1015.983470319631,42.28081050228295,9.342705479451775,141.9658675799087,0.5108675799086747,1.4029680365296804,11.183219178082192,NULL,NULL,NULL
stddev,NULL,NULL,8.25702272297828,9.449047807998806,9.050213032086162,18.302022402482653,0.434155160412567,29.971245179245873,NULL,0.07090278389602185,0.3525844445183553,9.186844318867593,3.941694571070647,115.49811650054748,8.023311356122313,44.7687773513328,1.6255092690049708,241.38845689299035,0.869199029899563,2.425779713613849,6.139908300420321,NULL,NULL,NULL
min,new York,2023-01-01T00:00:00,-0.1,-0.1,-0.1,16.18,0,0,rain,0,0,1.1,0,0,1000,0,0.5,0,0,0,10,Clear,clear-day,"72055399999,KJRB,..."
max,new York,2023-12-31T23:00:00,9.9,9.9,9.9,99.8,9.126,100,snow,6.1,8.2,9.8,9.9,99,999.9,99.9,9.9,992,4.2,9,8,"Snow, Rain, Parti...",snow,"KLGA,KNYC,7250301..."


### Checking for missing values

In [20]:
taxi_yellow_raw.select([sum(col(c).isNull().cast("int")).alias(c) for c in taxi_yellow_raw.columns]).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       0|                   0|                    0|        1237613|            0|   1237613|           1237613|           0|           0|           0|          0|    0|      0|         

In [21]:
taxi_green_raw.select([sum(col(c).isNull().cast("int")).alias(c) for c in taxi_green_raw.columns]).show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       0|                   0|                    0|             51289|     51289|           0|           0|          51289|            0|          0|    0|      

In [22]:
citybike_raw_data.select([sum(col(c).isNull().cast("int")).alias(c) for c in citybike_raw_data.columns]).show()

+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|      0|            0|         0|       0|             18141|           18141|           95626|         95626|        0|        0|  25047|  25047|            0|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+



In [23]:
weather_raw.select([sum(col(c).isNull().cast("int")).alias(c) for c in weather_raw.columns]).show()

+----+--------+----+---------+---+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+----------+----+--------+
|name|datetime|temp|feelslike|dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|conditions|icon|stations|
+----+--------+----+---------+---+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+----------+----+--------+
|   0|       0|   0|        0|  0|       0|     0|         0|      7766|   0|        0|       0|        0|      0|               0|         0|         0|             0|          0|      0|         0|         0|   0|       0|
+----+--------+----+---------+---+--------+------+----------+----------+----+---------+--------+----

In [24]:
# Convert the preciptype null in wether to not rain
weather_raw = weather_raw.withColumn("preciptype", 
                                       when(col("preciptype").isNull(), "not rain").otherwise(col("preciptype")))

## Pre-Processing

### combine the data of Yellow taxi and Green Taxi

In [25]:
taxi_yellow_raw.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [26]:
taxi_green_raw.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [27]:
# Unified Yellow taxi and Green taxi features
taxi_green_raw = taxi_green_raw.withColumnRenamed("lpep_pickup_datetime" ,"tpep_pickup_datetime")
taxi_green_raw = taxi_green_raw.withColumnRenamed("lpep_dropoff_datetime" ,"tpep_dropoff_datetime")
taxi_green_raw = taxi_green_raw.drop('ehail_fee').drop('trip_type')
taxi_yellow_raw = taxi_yellow_raw.drop('Airport_fee')

In [28]:
# Reorder the columns and join the two data sets
common_columns = ['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

taxi_yellow_raw = taxi_yellow_raw.select(common_columns)
taxi_green_raw = taxi_green_raw.select(common_columns)
taxi_raw_data = taxi_yellow_raw.union(taxi_green_raw)

In [29]:
taxi_raw_data.count()

35962309

In [30]:
# Remove the duplicated data of taxi
taxi_raw_data.dropDuplicates()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2023-10-01 00:29:26,2023-10-01 00:46:19,2,6.1,1,N,161,160,2,26.8,3.5,0.5,0.0,6.94,1.0,38.74,2.5
1,2023-10-01 00:48:56,2023-10-01 00:56:21,2,1.4,1,N,186,158,2,8.6,3.5,0.5,0.0,0.0,1.0,13.6,2.5
2,2023-10-01 00:53:01,2023-10-01 01:04:07,3,2.61,1,N,79,170,1,14.9,1.0,0.5,3.98,0.0,1.0,23.88,2.5
1,2023-10-01 00:25:11,2023-10-01 00:33:43,1,1.6,1,N,79,233,1,9.3,3.5,0.5,2.85,0.0,1.0,17.15,2.5
2,2023-10-01 00:06:02,2023-10-01 00:11:19,1,0.63,1,N,234,107,1,6.5,1.0,0.5,3.45,0.0,1.0,14.95,2.5
2,2023-10-01 00:16:59,2023-10-01 00:32:28,1,2.72,1,N,79,162,4,-17.0,-1.0,-0.5,0.0,0.0,-1.0,-22.0,-2.5
1,2023-10-01 00:36:26,2023-10-01 00:52:33,0,2.6,1,N,68,164,2,15.6,3.5,0.5,0.0,0.0,1.0,20.6,2.5
1,2023-10-01 00:56:37,2023-10-01 01:18:54,3,4.8,1,N,79,236,1,24.7,3.5,0.5,5.9,0.0,1.0,35.6,2.5
2,2023-10-01 00:24:28,2023-10-01 00:35:48,1,2.07,1,N,249,170,1,12.8,1.0,0.5,3.56,0.0,1.0,21.36,2.5
2,2023-10-01 00:42:07,2023-10-01 00:46:11,3,0.71,1,N,90,186,1,5.8,1.0,0.5,2.7,0.0,1.0,13.5,2.5


In [31]:
print(taxi_raw_data.count())
print(len(taxi_raw_data.columns))

35962309
18


### Unify weather time data with taxi time data and Citybike time data

In [4]:
weather_raw = weather_raw.withColumn("datetime", date_format(weather_raw['datetime'], 'yyyy-MM-dd HH:mm:ss'))

### Data Wrangling for taxi_raw_data

In [33]:
# Remove lines with passenger_count less than 0 or more than 6
taxi_data = taxi_raw_data.filter((col('passenger_count') > 0) & (col('passenger_count') <= 6))
# Remove lines with Rate Code ID greater than 6
taxi_data = taxi_data.filter((col('RateCodeID') <= 6))
# Remove lines with trip diatance smaller than 0
taxi_data = taxi_data.filter(col('trip_distance') > 0)
# Remove lines with payment type is 0
taxi_data = taxi_data.filter(col('payment_type') > 0)
# Remove the fees-related columns smaller than 0 
columns_to_check = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
                    'improvement_surcharge', 'total_amount']
for column in columns_to_check:
    taxi_data = taxi_data.filter(col(column) >= 0)
# remove the date not in range
taxi_data = taxi_data.filter((col('tpep_pickup_datetime') >= '2023-02-01') & 
                             (col('tpep_pickup_datetime') < '2024-01-01'))
taxi_data = taxi_data.filter((col('tpep_dropoff_datetime') >= '2023-02-01') & 
                             (col('tpep_dropoff_datetime') < '2024-01-01'))

In [34]:
# check the duration of take taxi
taxi_data = taxi_data.withColumn("duration", 
                (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60)
taxi_data.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "duration").describe()

summary,duration
count,33178211
mean,17.624698428334387
stddev,42.762921340188115
min,-58.55
max,7053.616666666667


In [35]:
#  remove the line with negative duration
taxi_data = taxi_data.filter(col('duration') > 0)

In [36]:
# calculate how much row has been moved
((taxi_raw_data.count()-taxi_data.count())/taxi_raw_data.count())*100

7.745562166211297

In [37]:
# the shap of taxi_data
print(taxi_data.count())
print(len(taxi_data.columns))

33176826
19


### Data Wrangling for cityBike_raw_data

In [38]:
# remove the data which not in New York
lat_min, lat_max = 40.4774, 40.9176
lng_min, lng_max = -74.2591, -73.7004

citybike_data = citybike_raw_data.filter(
    (col('start_lat').between(lat_min, lat_max)) &
    (col('start_lng').between(lng_min, lng_max)) &
    (col('end_lat').between(lat_min, lat_max)) &
    (col('end_lng').between(lng_min, lng_max))
)
citybike_data = citybike_data.filter((col('started_at') >= '2023-02-01') & 
                             (col('started_at') < '2024-01-01'))
citybike_data = citybike_data.filter((col('ended_at') >= '2023-02-01') & 
                             (col('ended_at') < '2024-01-01'))

In [39]:
# check the duration of citiBike data
citybike_data = citybike_data.withColumn("duration", 
                (unix_timestamp("ended_at") - unix_timestamp("started_at")) / 60)

In [40]:
#  remove the line with negative duration
citybike_data = citybike_data.filter(col('duration') > 0)

In [41]:
(citybike_raw_data.count()-citybike_data.count())/citybike_raw_data.count()*100

0.07682304714229131

In [42]:
# the shap of citybike_data
print(citybike_data.count())
print(len(citybike_data.columns))

33286027
14


### Data Wrangling for weather_raw

In [5]:
weather_raw = weather_raw.filter((col('datetime') >= '2023-02-01') & 
                             (col('datetime') < '2024-01-01'))

In [6]:
print(weather_raw.count())
print(len(weather_raw.columns))

8016
24


24/08/25 03:24:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1614029 ms exceeds timeout 120000 ms
24/08/25 03:24:13 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/25 03:24:19 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

## Save data Which after preprocessing

In [44]:
taxi_data.write.parquet('../data/data_after_preprocessing/taxi')
weather_raw.write.parquet('../data/data_after_preprocessing/weather')
citybike_data.write.parquet('../data/data_after_preprocessing/citybike')